### Hyperpot

Regular training local with hyper parameter tuning using Hyperpot and Vizier

In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import os
for dirname, _, filenames in os.walk('datasets/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

datasets/Wholesale customers data.csv


In [21]:
df = pd.read_csv(f'datasets/{filename}')
df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867
438,1,3,10290,1981,2232,1038,168,2125


In [22]:
# Drops

X = df.drop('Channel', axis=1)
y = df['Channel']

In [23]:
# Convert labels into binary values

y[y == 2] = 0
y[y == 1] = 1

In [24]:
# Split data for training

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [55]:
space={
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 1,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': 180,
    'seed': 0
    }

In [61]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], 
                    max_depth = int(space['max_depth']), 
                    gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),
                    min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]

    xgb_params = {}
    xgb_params['eval_metric']='auc'
    xgb_params['early_stopping_rounds']=10 

    clf.set_params(**xgb_params)
    
    clf.fit(
        X_train, 
        y_train,
        eval_set=evaluation, 
        verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [57]:

xgb_params = {}
xgb_params['eval_metric']='auc'
xgb_params['early_stopping_rounds']=10 
xgb_params

{'eval_metric': 'auc', 'early_stopping_rounds': 10}

In [58]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

SCORE:                                                 
0.3484848484848485                                     
SCORE:                                                                            
0.3484848484848485                                                                
SCORE:                                                                            
0.3484848484848485                                                                
SCORE:                                                                            
0.8712121212121212                                                                
SCORE:                                                                            
0.8863636363636364                                                                
SCORE:                                                                            
0.3484848484848485                                                                
SCORE:                                                    

In [59]:
print(best_hyperparams)

{'colsample_bytree': 0.8015834851378087, 'gamma': 4.160392182471469, 'max_depth': 9.0, 'min_child_weight': 7.0, 'reg_alpha': 74.0, 'reg_lambda': 0.8708226877745266}


In [64]:
colsample_bytree=best_hyperparams["colsample_bytree"]
max_depth=best_hyperparams["max_depth"]
gamma=best_hyperparams["gamma"]
reg_alpha=best_hyperparams["reg_alpha"]
min_child_weight=best_hyperparams["min_child_weight"]
colsample_bytree=best_hyperparams["colsample_bytree"]

clf=xgb.XGBClassifier(
    n_estimators =180, 
    max_depth = int(max_depth), 
    gamma = gamma,
    reg_alpha = reg_alpha,
    min_child_weight=min_child_weight,
    colsample_bytree=colsample_bytree)

evaluation = [( X_train, y_train), ( X_test, y_test)]

clf.fit(
    X_train, 
    y_train,
    eval_set=evaluation, 
    verbose=False)

pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, pred>0.5)
print ("SCORE:", accuracy)

SCORE: 0.8939393939393939


In [66]:
clf=xgb.XGBClassifier()

evaluation = [( X_train, y_train), ( X_test, y_test)]

clf.fit(
    X_train, 
    y_train,
    eval_set=evaluation, 
    verbose=False)

pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, pred>0.5)
print ("SCORE:", accuracy)

SCORE: 0.8939393939393939


In [68]:
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,0,3,12669,9656,7561,214,2674,1338
1,0,3,7057,9810,9568,1762,3293,1776
2,0,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,0,3,22615,5410,7198,3915,1777,5185


In [88]:
# Prediction Single Row

x=pd.DataFrame(X_test.iloc[1,:]).T
pred=clf.predict(x)
pred

array([0])

## Using Vertex Vizier

Standalone, vizier only gives you recommendations about the next trial but doesn't run the trial for you, for that we'll create a separate section called Using Vertex HPT+Vizier

In [9]:
import datetime

PROJECT_ID = "jchavezar-demo"
REGION = "us-central1"
STUDY_DISPLAY_NAME = "{}_study_{}".format(
    PROJECT_ID.replace("-", ""), datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
ENDPOINT = REGION + "-aiplatform.googleapis.com"
PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

In [10]:
print("ENDPOINT: {}".format(ENDPOINT))
print("REGION: {}".format(REGION))
print("PARENT: {}".format(PARENT))

ENDPOINT: us-central1-aiplatform.googleapis.com
REGION: us-central1
PARENT: projects/jchavezar-demo/locations/us-central1


In [11]:
# Parameter Configuration

max_depth = {"parameter_id": "max_depth", "integer_value_spec": {"min_value": 3, "max_value": 18}}
gamma = {"parameter_id": "gamma", "integer_value_spec": {"min_value": 1, "max_value": 9}}
reg_alpha = {"parameter_id": "reg_alpha", "integer_value_spec": {"min_value": 1, "max_value": 9}}
reg_lambda = {"parameter_id": "reg_lambda", "integer_value_spec": {"min_value": 0, "max_value": 1}}
colsample_bytree = {"parameter_id": "colsample_bytree", "double_value_spec": {"min_value": 0.5, "max_value": 1.0}}
min_child_weight = {"parameter_id": "min_child_weight", "integer_value_spec": {"min_value": 0, "max_value": 10.0}}

metrics = {"metric_id": "accuracy", "goal": "MAXIMIZE"}

study = {
    "display_name": STUDY_DISPLAY_NAME,
    "study_spec": {
        "algorithm": "RANDOM_SEARCH",
        "parameters": [
            max_depth,
            gamma,
            reg_alpha,
            reg_lambda,
            colsample_bytree,
            min_child_weight
        ],
        "metrics": [metrics],
    },
}

In [12]:
import json

print(json.dumps(study, indent=2, sort_keys=True))

{
  "display_name": "jchavezardemo_study_20220803_105855",
  "study_spec": {
    "algorithm": "RANDOM_SEARCH",
    "metrics": [
      {
        "goal": "MAXIMIZE",
        "metric_id": "accuracy"
      }
    ],
    "parameters": [
      {
        "integer_value_spec": {
          "max_value": 18,
          "min_value": 3
        },
        "parameter_id": "max_depth"
      },
      {
        "integer_value_spec": {
          "max_value": 9,
          "min_value": 1
        },
        "parameter_id": "gamma"
      },
      {
        "integer_value_spec": {
          "max_value": 9,
          "min_value": 1
        },
        "parameter_id": "reg_alpha"
      },
      {
        "integer_value_spec": {
          "max_value": 1,
          "min_value": 0
        },
        "parameter_id": "reg_lambda"
      },
      {
        "double_value_spec": {
          "max_value": 1.0,
          "min_value": 0.5
        },
        "parameter_id": "colsample_bytree"
      },
      {
        "integer_v

In [13]:
from google.cloud import aiplatform

vizier_client = aiplatform.gapic.VizierServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)
study = vizier_client.create_study(parent=PARENT, study=study)
STUDY_ID = study.name
print("STUDY_ID: {}".format(STUDY_ID))

STUDY_ID: projects/569083142710/locations/us-central1/studies/2824667673603


In [17]:
import xgboost as xgb

def objective(trial_id, max_depth, gamma, reg_alpha, reg_lambda, colsample_bytree, min_child_weight):
    print(("=========== Start Trial: [{}] =============").format(trial_id))
    clf=xgb.XGBClassifier(
                    n_estimators =180, 
                    max_depth = max_depth, 
                    gamma = gamma,
                    reg_alpha = reg_alpha,
                    min_child_weight= min_child_weight,
                    colsample_bytree= colsample_bytree)
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]

    xgb_params = {}
    xgb_params['eval_metric']='auc'
    xgb_params['early_stopping_rounds']=10 

    clf.set_params(**xgb_params)
    
    clf.fit(
        X_train, 
        y_train,
        eval_set=evaluation, 
        verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    metric = {"metric_id": "accuracy", "value": accuracy}
    print ("SCORE:", accuracy)
    return [metric]

In [18]:
client_id = "client1"  # @param {type: 'string'}
suggestion_count_per_request = 5  # @param {type: 'integer'}
max_trial_id_to_stop = 4  # @param {type: 'integer'}

print("client_id: {}".format(client_id))
print("suggestion_count_per_request: {}".format(suggestion_count_per_request))
print("max_trial_id_to_stop: {}".format(max_trial_id_to_stop))

client_id: client1
suggestion_count_per_request: 5
max_trial_id_to_stop: 4


In [25]:
trial_id = 0
while int(trial_id) < max_trial_id_to_stop:
    suggest_response = vizier_client.suggest_trials(
        {
            "parent": STUDY_ID,
            "suggestion_count": suggestion_count_per_request,
            "client_id": client_id,
        }
    )

    for suggested_trial in suggest_response.result().trials:
        trial_id = suggested_trial.name.split("/")[-1]
        trial = vizier_client.get_trial({"name": suggested_trial.name})

        if trial.state in ["COMPLETED", "INFEASIBLE"]:
            continue

        for param in trial.parameters:
            if param.parameter_id == "max_depth":
                max_depth = int(param.value)
            elif param.parameter_id == "gamma":
                gamma = int(param.value)
            elif param.parameter_id == "reg_alpha":
                reg_alpha = int(param.value)
            elif param.parameter_id == "reg_lambda":
                reg_lambda = int(param.value)
            elif param.parameter_id == "colsample_bytree":
                colsample_bytree = param.value
            elif param.parameter_id == "min_child_weight":
                min_child_weight = int(param.value)        
        print("""
        Trial : max_depth is {}, 
        gamma is {}, 
        reg_alpha is {}, 
        reg_lambda is {}, 
        colsample_bytree is {},
        min_child_weight is {}
        """.format(max_depth, gamma, reg_alpha, reg_lambda, colsample_bytree, min_child_weight))

        vizier_client.add_trial_measurement(
            {
                "trial_name": suggested_trial.name,
                "measurement": {
                    "metrics": objective(suggested_trial.name, max_depth, gamma, reg_alpha, reg_lambda, colsample_bytree, min_child_weight)
                },
            }
        )

        response = vizier_client.complete_trial(
            {"name": suggested_trial.name, "trial_infeasible": False}
        )


        Trial : max_depth is 15, 
        gamma is 8, 
        reg_alpha is 6, 
        reg_lambda is 0, 
        colsample_bytree is 0.5452238771161251,
        min_child_weight is 9
        
=========== Start Trial: [projects/569083142710/locations/us-central1/studies/2824667673603/trials/1] =============
SCORE: 0.8863636363636364

        Trial : max_depth is 18, 
        gamma is 1, 
        reg_alpha is 9, 
        reg_lambda is 1, 
        colsample_bytree is 0.6815091398605306,
        min_child_weight is 9
        
=========== Start Trial: [projects/569083142710/locations/us-central1/studies/2824667673603/trials/2] =============
SCORE: 0.8939393939393939

        Trial : max_depth is 17, 
        gamma is 3, 
        reg_alpha is 1, 
        reg_lambda is 0, 
        colsample_bytree is 0.5358641343776482,
        min_child_weight is 8
        
=========== Start Trial: [projects/569083142710/locations/us-central1/studies/2824667673603/trials/3] =============
SCORE: 0.886363636

In [28]:
# List Optimal Trials

optimal_trials = vizier_client.list_optimal_trials({"parent": STUDY_ID})

print("optimal_trials: {}".format(optimal_trials))

optimal_trials: optimal_trials {
  name: "projects/569083142710/locations/us-central1/studies/2824667673603/trials/2"
  state: SUCCEEDED
  parameters {
    parameter_id: "colsample_bytree"
    value {
      number_value: 0.6815091398605306
    }
  }
  parameters {
    parameter_id: "gamma"
    value {
      number_value: 1.0
    }
  }
  parameters {
    parameter_id: "max_depth"
    value {
      number_value: 18.0
    }
  }
  parameters {
    parameter_id: "min_child_weight"
    value {
      number_value: 9.0
    }
  }
  parameters {
    parameter_id: "reg_alpha"
    value {
      number_value: 9.0
    }
  }
  parameters {
    parameter_id: "reg_lambda"
    value {
      number_value: 1.0
    }
  }
  final_measurement {
    metrics {
      metric_id: "accuracy"
      value: 0.8939393939393939
    }
  }
  measurements {
    metrics {
      metric_id: "accuracy"
      value: 0.8939393939393939
    }
  }
  start_time {
    seconds: 1659538745
  }
  end_time {
    seconds: 1659538789
  }

In [7]:
# Clean


vizier_client.delete_study({"name": "projects/569083142710/locations/us-central1/studies/1176465079566"})

In [4]:
from google.cloud import aiplatform

ENDPOINT = "us-central1" + "-aiplatform.googleapis.com"


vizier_client = aiplatform.gapic.VizierServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

## Using Vertex HPT+Vizier

### Variables

In [229]:
MODEL_DIR = "gs://vtx-models/hpt-model"
IMAGE_URI = "us-central1-docker.pkg.dev/jchavezar-demo/trainings/train_hpt_xgb:latest"

### Package Assembly

In [163]:
# Make folder for Python hyperparameter tuning script
! rm -rf custom
! mkdir custom

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

In [252]:
%%writefile custom/trainer/task.py

import os
import argparse
import datetime
import logging
import pandas as pd
import xgboost as xgb
from hypertune import HyperTune
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from google.cloud import storage

def make_dataset(dataset_uri):
    df = pd.read_csv(dataset_uri)
    X = df.drop('Channel', axis=1)
    y = df['Channel']
    y[y == 2] = 0
    y[y == 1] = 1

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    return X_train, X_test, y_train, y_test

def create_model( 
    max_depth, 
    gamma, 
    reg_alpha, 
    colsample_bytree, 
    min_child_weight,
    ):
    
    print(("=========== Start Trial: [] ============="))
    clf=xgb.XGBClassifier(
                    n_estimators =180,
                    max_depth = max_depth, 
                    gamma = gamma,
                    reg_alpha = reg_alpha,
                    min_child_weight= min_child_weight,
                    colsample_bytree= colsample_bytree)
    
    xgb_params = {}
    xgb_params['eval_metric']='auc'
    xgb_params['early_stopping_rounds']=10 

    clf.set_params(**xgb_params)
    
    return clf


def main():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--dataset_dir',
                    type=str, 
                    help='Dataset dir.')
    parser.add_argument('--model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), 
                    type=str, 
                    help='Model dir.')
    parser.add_argument('--max_depth',
                    default=3, type=int,
                    help='Maximum tree depth.')
    parser.add_argument('--gamma',
                    default=1, type=int,
                    help='Tree gamma.')
    parser.add_argument('--reg_alpha',
                    default=1, type=int,
                    help='Alpha Reg.')
    parser.add_argument('--colsample_bytree',
                    default=0.5, type=float,
                    help='Colum sample tree depth.')
    parser.add_argument('--min_child_weight',
                    default=0, type=int,
                    help='Colum sample tree depth.')
    args = parser.parse_args()

    X_train, X_test, y_train, y_test = make_dataset(args.dataset_dir)
    print(X_train.head(2))

    model = create_model(args.max_depth, args.gamma, args.reg_alpha, args.colsample_bytree, args.min_child_weight)
    evaluation = [(X_train, y_train), (X_test, y_test)]
    model.fit(X_train, y_train, eval_set=evaluation, verbose=False)
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    metric = {"metric_id": "accuracy", "value": accuracy}
    print ("SCORE:", accuracy)

    hpt = HyperTune()
    hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag='accuracy',
    metric_value=accuracy,
    global_step=1)
    
    # GCSFuse conversion
    gs_prefix = 'gs://'# Export the classifier to a file
    gcsfuse_prefix = '/gcs/'
    if args.model_dir.startswith(gs_prefix):
        args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
        dirpath = os.path.split(args.model_dir)[0]
        if not os.path.isdir(dirpath):
            os.makedirs(dirpath)

    print(args.model_dir)

    # Export the classifier to a file
    gcs_model_path = os.path.join(args.model_dir, 'model.bst')
    logging.info("Saving model artifacts to {}". format(gcs_model_path))
    model.save_model(gcs_model_path)

if __name__ == "__main__":
    main()

Overwriting custom/trainer/task.py


In [253]:
%%writefile custom/requirements.txt
cloudml-hypertune
xgboost
pandas
gcsfs
sklearn
google-cloud-storage

Overwriting custom/requirements.txt


In [254]:
%%writefile custom/Dockerfile
FROM python:3.7

WORKDIR /

COPY trainer /trainer
COPY requirements.txt requirements.txt

# Installs hypertune library
RUN pip install -r requirements.txt


# Copies the trainer code to the docker image.print(
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Overwriting custom/Dockerfile


In [255]:
!gcloud builds submit -t $IMAGE_URI custom/.

Creating temporary tarball archive of 4 file(s) totalling 3.8 KiB before compression.
Uploading tarball of [custom/.] to [gs://jchavezar-demo_cloudbuild/source/1659637393.979326-cb9d5bfd64824deeb9597e709509578f.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/67a76bea-8484-4f6e-afe2-c47031e27d40].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/67a76bea-8484-4f6e-afe2-c47031e27d40?project=569083142710].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "67a76bea-8484-4f6e-afe2-c47031e27d40"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1659637393.979326-cb9d5bfd64824deeb9597e709509578f.tgz#1659637394373831
Copying gs://jchavezar-demo_cloudbuild/source/1659637393.979326-cb9d5bfd64824deeb9597e709509578f.tgz#1659637394373831...
/ [1 files][  1.6 KiB/  1.6 KiB]                                                
Operation completed over 1 obje

In [256]:
import google.cloud.aiplatform as aip
from google.cloud.aiplatform import hyperparameter_tuning as hpt

aip.init(project="jchavezar-demo", staging_bucket="gs://vtx-staging/")

CMDARGS = [
    "--dataset_dir=gs://vtx-datasets-public/wholesale_data.csv"
]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": {"machine_type":"n1-standard-4", "accelerator_count":0},
        "container_spec": {
            "image_uri": IMAGE_URI,
            "args": CMDARGS,
        },
    }
]
job = aip.CustomJob(
    display_name="xgboost_hpt", 
    worker_pool_specs=worker_pool_spec,
)

hpt_job = aip.HyperparameterTuningJob(
    display_name="xgboost_hpt",
    custom_job=job,
    metric_spec={
        "accuracy": "maximize",
    },
    parameter_spec={
        "max_depth": hpt.IntegerParameterSpec(min=3, max=18, scale="linear"),
        "gamma": hpt.IntegerParameterSpec(min=1, max=9, scale="linear"),
        "reg_alpha": hpt.IntegerParameterSpec(min=1, max=9, scale="linear"),
        "colsample_bytree": hpt.DoubleParameterSpec(min=0.5, max=1, scale="log"),
        "min_child_weight": hpt.IntegerParameterSpec(min=1, max=9, scale="linear"),
    },
    search_algorithm="random",
    max_trial_count=4,
    parallel_trial_count=4,
)


In [257]:
hpt_job.run()

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/569083142710/locations/us-central1/hyperparameterTuningJobs/4514033168084369408
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/569083142710/locations/us-central1/hyperparameterTuningJobs/4514033168084369408')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4514033168084369408?project=569083142710
HyperparameterTuningJob projects/569083142710/locations/us-central1/hyperparameterTuningJobs/4514033168084369408 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/569083142710/locations/us-central1/hyperparameterTuningJobs/4514033168084369408 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/569083142710/locations/us-central1/hyperparameterTuningJobs/4514033168084369408 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJ

In [278]:
best = (None, None, None, None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[6]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.parameters[2].value),
                float(trial.parameters[3].value),
                float(trial.parameters[4].value),
                float(trial.final_measurement.metrics[0].value),
            )
            print(best)
        except:
            best = (
                trial.id,
                None,
                None,
                None,
                None,
                None,
                float(trial.final_measurement.metrics[0].value),
            )


('1', 0.9580971742324845, 8.0, 12.0, 6.0, 2.0, 0.8939393939393939)


In [289]:
# Best Model is under vtx-staging/custom-job-date/trial_id:

!gsutil ls gs://vtx-staging/aiplatform-custom-job-2022-08-04-14:26:28.960/1/model

gs://vtx-staging/aiplatform-custom-job-2022-08-04-14:26:28.960/1/model/
gs://vtx-staging/aiplatform-custom-job-2022-08-04-14:26:28.960/1/model/model.bst
